In [22]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [23]:
import pandas as pd
file_name='finance_data/Sentences_AllAgree.txt'
f = open(file_name,encoding='latin1')
data=[]
for line in f:
        line = line.replace('\n', '')
        data.append(line.split('@'))
data=pd.DataFrame(data,columns=['contenu','target'])

In [26]:
data

,contenu,target
0,"According to Gran , the company has no plans t...",neutral
1,"For the last quarter of 2010 , Componenta 's n...",positive
2,"In the third quarter of 2010 , net sales incre...",positive
3,Operating profit rose to EUR 13.1 mn from EUR ...,positive
4,"Operating profit totalled EUR 21.1 mn , up fro...",positive
...,...,...
2259,Operating result for the 12-month period decre...,negative
2260,HELSINKI Thomson Financial - Shares in Cargote...,negative
2261,LONDON MarketWatch -- Share prices ended lower...,negative
2262,Operating profit fell to EUR 35.4 mn from EUR ...,negative


In [24]:
wordcloud = WordCloud(random_state=8)

In [27]:
text=' '.join(i for i in data.contenu)
wordcloud.generate(text)
plt.figure(figsize=(15,10))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()
text

ValueError: Only supported for TrueType fonts

In [29]:
# right the top 20 stopwords in the wordcloud dic
print(list(WordCloud().stopwords)[:20])

["they'd", 'how', "he'd", "they've", 'both', 'this', 'we', 'which', 'but', 'against', 'again', 'some', 'get', "wouldn't", "couldn't", 'hers', 'also', 'from', "i'm", 'cannot']


In [30]:
data.iloc[-2].contenu    

'Operating profit fell to EUR 35.4 mn from EUR 68.8 mn in 2007 , including vessel sales gain of EUR 12.3 mn .'

In [32]:
import numpy as np
y=data.target.astype(str)
punct_pattern=r'[\!"#$%&\*+,-./:;?@^_`()|~]'
data.loc[:,'contenu_no_puct']=data.contenu.str.replace(punct_pattern,'')




In [33]:
data.iloc[-2].contenu_no_puct    

'Operating profit fell to EUR 35.4 mn from EUR 68.8 mn in 2007 , including vessel sales gain of EUR 12.3 mn .'

In [34]:
import re 
# remove numbers from data set
digitspattern = r'[0-9]'

data.loc[:,'contenu_no_num_no_punc']=data.contenu_no_puct.str.replace(digitspattern,'')



In [35]:
data.iloc[-2].contenu_no_num_no_punc

'Operating profit fell to EUR 35.4 mn from EUR 68.8 mn in 2007 , including vessel sales gain of EUR 12.3 mn .'

In [36]:
wordcloud.generate(' '.join(i for i in data.contenu_no_num_no_punc))
plt.figure(figsize=(15,10))
plt.imshow(wordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()

ValueError: Only supported for TrueType fonts

In [37]:
# we will remove 'country's name (finnish is frequent here) 

In [38]:

spacepattern=r'\s\s+'
data.loc[:,'contenu_no_num_no_punc']=data.contenu_no_num_no_punc.str.replace(spacepattern,' ')



In [39]:
data.contenu_no_num_no_punc[3]

'Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing 7.7 % of net sales .'

In [40]:
data.contenu[3]

'Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing 7.7 % of net sales .'

In [41]:
wordcloud.generate(' '.join(i for i in data.contenu_no_num_no_punc))
plt.figure(figsize=(15,10))
plt.imshow(wordcloud,interpolation='bilinear')
plt.axis('off')
plt.show()

ValueError: Only supported for TrueType fonts

In [42]:
# transform label into numbers
data.loc[data['target']=='neutral','target']=0
data.loc[data['target']=='positive','target']=1
data.loc[data['target']=='negative','target']=2

y=data.target.astype(int)

#### Vectorize the text we would use Sckitis-learn has three types of vectorization 
###### we would work with CountVectorizer and TF-IDF

In [43]:
import spacy 

nlp=spacy.load('en_core_web_sm')

In [44]:
def lemmatize(text):
    doc=nlp(text)   
    tokens=[token.lemma_ for token in doc if not (token.is_stop or token.is_punct or len(token.lemma_)<=2)]

    return ' '.join(tokens)
def POS(text):
    doc=nlp(text)   
    pos=[token.pos_ for token in doc if not (token.is_stop or len(token.lemma_)<=2)]

    return ' '.join(pos)
def NER(text):
    doc=nlp(text)   

    tokens=[token.lemma_ for token in doc if not (token.is_stop )]

    ner=[]
    for i in tokens:
        j=False
        for ent in doc.ents:
            if i in ent.text:
                ner.append(ent.label_)
                j=True

        if j==False:
            ner.append(i)
    return ' '.join(ner)
        

In [45]:
data['processed_text']=data.contenu_no_num_no_punc.apply(lambda txt : lemmatize(txt))


In [46]:
data['pos']=data.contenu_no_num_no_punc.apply(lambda txt : POS(txt))


In [47]:
data['ner']=data.contenu_no_num_no_punc.apply(lambda txt : NER(txt))

In [48]:
data['ner']

0       accord PERSON , company plan production GPE , ...
1       DATE DATE , ORG net sale double PERSON ORG DAT...
2       DATE DATE , net sale increase PERCENT PERCENT ...
3       operate profit rise ORG ORG CARDINAL mn ORG OR...
4       operate profit total PRODUCT ORG PRODUCT mn , ...
                              ...                        
2259    operate result DATE DATE DATE period decrease ...
2260    ORG ORG ORG ORG DATE ORG GPE fall sharply TIME...
2261    ORG ORG -- share price end lower GPE DATE rebo...
2262    operate profit fall ORG ORG ORG CARDINAL mn OR...
2263    sale GPE GPE decrease PERCENT PERCENT PERCENT ...
Name: ner, Length: 2264, dtype: object

In [49]:
def VERB_ADJ(text):
    doc=nlp(text)   
    pos=[token.lemma_.lower() for token in doc if  (token.is_stop==False and (token.pos_=='VERB'))]
    return ' '.join(pos)


In [50]:
#data['ner']=data.contenu_no_num_no_punc.apply(lambda txt : VERB_ADJ(txt))

In [51]:
# The CountVectorizer counts the word occurrences in each document.
from sklearn.feature_extraction.text import CountVectorizer
#from nltk.tokenize import RegexpTokenizer

cv = CountVectorizer(stop_words='english')
X=cv.fit_transform((data.processed_text))
print(X.shape)
print(type(X))
vocab = cv.get_feature_names_out()
docterm = pd.DataFrame(X.todense(), columns=vocab)


(2264, 4971)
<class 'scipy.sparse._csr.csr_matrix'>


In [52]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=4)
X_train.shape

(1811, 4971)

In [53]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
clf=MultinomialNB()

clf.fit(X_train,y_train)

yhat = clf.predict(X_train)
print("Accuracy: ",accuracy_score(y_train, yhat))

Accuracy:  0.9166206515737162


In [54]:
yhat = clf.predict(X_test)
print("Accuracy: ",accuracy_score(y_test, yhat))

Accuracy:  0.7969094922737306


In [55]:
# use TfidfVectorizer

In [56]:
# import and instantiate the vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

# apply the vectorizer to the corpus
X = vectorizer.fit_transform(data.ner)

# display the document-term matrix
vocab = vectorizer.get_feature_names_out()
docterm = pd.DataFrame(X.todense(), columns=vocab)

In [57]:
docterm

,00,000,0030,007,01,02,03,031,04,05,...,âm,ârbindelsen,än,ñhikauppa,ñinen,ñl,ñlt,ñnnen,ñrnit,ñrvi
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2259,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2261,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2262,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=4)
X_train.shape

(1811, 3219)

In [59]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
clf=MultinomialNB()

clf.fit(X_train,y_train)

yhat = clf.predict(X_train)
print("Accuracy: ",accuracy_score(y_train, yhat))

Accuracy:  0.792932081722805


In [60]:
yhat = clf.predict(X_test)
print("Accuracy: ",accuracy_score(y_test, yhat))

Accuracy:  0.7373068432671082


In [79]:
# The CountVectorizer counts the word occurrences in each document.
from sklearn.feature_extraction.text import CountVectorizer
#from nltk.tokenize import RegexpTokenizer

cv = CountVectorizer(stop_words='english')
X=cv.fit_transform((data.ner))
print(X.shape)
print(type(X))
vocab = cv.get_feature_names_out()
docterm = pd.DataFrame(X.todense(), columns=vocab)


(2264, 3183)
<class 'scipy.sparse._csr.csr_matrix'>


In [80]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=4)
X_train.shape

(1811, 3183)

In [82]:
from sklearn.metrics import recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from sklearn.naive_bayes import GaussianNB


ada_classifier = AdaBoostClassifier()

dt_classifier = DecisionTreeClassifier()

randfor_classifier = RandomForestClassifier()


fit_ada = ada_classifier.fit(X_train, y_train)
fit_dt = dt_classifier.fit(X_train, y_train)
fit_gnb = randfor_classifier.fit(X_train, y_train)  # GaussianNB attend des matrices denses

pred_ada = fit_ada.predict(X_test)
pred_dt = fit_dt.predict(X_test)
pred_gnb = fit_gnb.predict(X_test)

probabilities_ada = fit_ada.predict_proba(X_test)[:, 1]
print('AdaBoostClassifier',accuracy_score(y_test, pred_ada))
print('DecisionTreeClassifier',accuracy_score(y_test, pred_dt))
print('GaussianNB',accuracy_score(y_test, pred_gnb))


AdaBoostClassifier 0.8410596026490066
DecisionTreeClassifier 0.8587196467991169
GaussianNB 0.869757174392936


In [65]:
from sklearn.metrics import recall_score, f1_score
recall_ada = recall_score(y_test, pred_ada,average='weighted')
f1_ada=f1_score(y_test, pred_ada,average='weighted')
recall_dt = recall_score(y_test, pred_dt,average='weighted')
f1_dt=f1_score(y_test, pred_dt,average='weighted')
recall_gnb =recall_score(y_test, pred_gnb,average='weighted')
f1_gnb=f1_score(y_test, pred_gnb,average='weighted')


In [66]:
print("Comparaison des modèles :")
print("AdaBoostClassifier :")
print("Recall (Rappel) : {:.2f}%".format(recall_ada * 100))
print("F1-Score : {:.2f}%".format(f1_ada * 100))
print("\nDecisionTreeClassifier :")
print("Recall (Rappel) : {:.2f}%".format(recall_dt * 100))
print("F1-Score : {:.2f}%".format(f1_dt * 100))
print("\nGaussianNB :")
print("Recall (Rappel) : {:.2f}%".format(recall_gnb * 100))
print("F1-Score : {:.2f}%".format(f1_gnb * 100))

Comparaison des modèles :
AdaBoostClassifier :
Recall (Rappel) : 84.11%
F1-Score : 83.46%

DecisionTreeClassifier :
Recall (Rappel) : 84.99%
F1-Score : 84.78%

GaussianNB :
Recall (Rappel) : 85.43%
F1-Score : 84.62%


In [67]:
pridct=['Sales hit record high last quarter','Operating profit fell to 10 EUR   from 20 EUR   in 2007   including vessel sales gain of EUR 12 3 mn ','sales decrease last quarter','i went to the shop','more efficient in workplace','Congratulations! you have won  Walmart gift card','profit rise from 100 to 2000']

ner_pred=[]
for i in pridct:
    ner_pred.append(NER(i))
ner_pred

['sale hit record high DATE',
 'operate profit fall CARDINAL ORG ORG ORG    CARDINAL DATE ORG ORG ORG    DATE    include vessel sale gain ORG ORG ORG CARDINAL CARDINAL mn',
 'sale decrease DATE',
 'go shop',
 'efficient workplace',
 'congratulation ! win   ORG gift card',
 'profit rise CARDINAL CARDINAL']

In [68]:
prd=cv.transform(ner_pred)

In [84]:
pred=fit_dt.predict(prd)
pred

array([1, 2, 2, 0, 0, 1, 1])

In [73]:
import pickle
with open('model.pk','wb') as f:
    pickle.dump(fit_dt,f)

In [75]:
with open('model.pk','rb') as f:
    model=pickle.load(f)

In [76]:
model.predict(prd)

array([1, 2, 2, 0, 1, 1, 1])

### save CounterVectorizer 

In [77]:
with open('CounterVectorizer.pk','wb') as f:
    pickle.dump(cv,f)

In [78]:
with open('CounterVectorizer.pk','rb') as f:
    model=pickle.load(f)